Name: Bakri Badet  
Matrikelnr:1125983

In [48]:
import recordlinkage as rl
from recordlinkage.preprocessing import clean
import pandas as pd




In [49]:
# ********************************
# Load Data
# ********************************
goog = pd.read_parquet('data/Google')
amzn = pd.read_parquet('data/Amazon')

In [50]:
goog.head()

,id,name,description,manufacturer,price
0,http://www.google.com/base/feeds/snippets/1112...,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,http://www.google.com/base/feeds/snippets/1153...,superstart! fun with reading & writing!,fun with reading & writing! is designed to hel...,,8.49
2,http://www.google.com/base/feeds/snippets/1134...,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for re...,intuit,637.99
3,http://www.google.com/base/feeds/snippets/1204...,math missions: the amazing arcade adventure (g...,save spectacle city by disrupting randall unde...,,12.95
4,http://www.google.com/base/feeds/snippets/1224...,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99


In [51]:
amzn.head()

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


In [52]:
print(goog.dtypes)
print(amzn.dtypes)

id              object
name            object
description     object
manufacturer    object
price           object
dtype: object
id               object
title            object
description      object
manufacturer     object
price           float64
dtype: object


In [53]:
goog.columns

Index(['id', 'name', 'description', 'manufacturer', 'price'], dtype='object')

In [54]:
amzn.columns

Index(['id', 'title', 'description', 'manufacturer', 'price'], dtype='object')

In [55]:
# ********************************
# Pre-processing
# ********************************
goog['name'] = clean(goog['name'])
goog['description'] = clean(goog['description'])
goog['manufacturer'] = clean(goog['manufacturer'])
goog['price'] = pd.to_numeric(goog['price'], errors='coerce')
goog.columns = ['idGoogle', 'name', 'description', 'manufacturer', 'price']
goog = goog.set_index('idGoogle')

amzn['title'] = clean(amzn['title'])
amzn['description'] = clean(amzn['description'])
amzn['manufacturer'] = clean(amzn['manufacturer'])
amzn.columns = ['idAmazon', 'name', 'description', 'manufacturer', 'price']
amzn = amzn.set_index('idAmazon')

In [56]:
amzn.head()

,name,description,manufacturer,price
idAmazon,,,,
b000jz4hqo,clickart 950 000 premier image pack,,broderbund,0.00
b0006zf55o,ca international arcserve lapdesktop oem 30pk,oem arcserve backup v111 win 30u for laptops a...,computer associates,0.00
b00004tkvy,noahs ark activity center,,victory multimedia,0.00
b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
b0006se5bq,singing coach unlimited,singing coach unlimited electronic learning pr...,carry a tune technologies,99.99


In [57]:
amzn.index

Index(['b000jz4hqo', 'b0006zf55o', 'b00004tkvy', 'b000g80lqo', 'b0006se5bq',
       'b000ehpzv8', 'b00021xhzw', 'b000gzwjgc', 'b0000dbykm', 'b00029bqa2',
       ...
       'b0009x6qew', 'b000tdgyks', 'b000pihtbk', 'b0002yiuns', 'b000in8mj0',
       'b000cs3s2c', 'b00005bigp', 'b000h1df7w', 'b000p9cr66', 'b000j588g4'],
      dtype='object', name='idAmazon', length=1363)

In [128]:
from recordlinkage import preprocessing
goog["phonetic_name"]=preprocessing.phonetic(goog["manufacturer"],"soundex")
amzn["phonetic_name"]=preprocessing.phonetic(amzn["manufacturer"],"soundex")

In [129]:
from recordlinkage import Index
indexer=Index()
indexer.block("phonetic_name")
# block macht index for Sample not the whole dataset
# Indexing reduces the number of record pairs to be compared. A common approach is blocking
# on a common attribute
candidate_links=indexer.index(goog,amzn)
# candidate_links.shape

In [130]:
candidate_links_df=candidate_links.to_frame(index=False,name=["idGoogle","idAmazon"])
# candidate_links_df[candidate_links_df["idAmazon"]=="b000hc0m36"]
candidate_links_df

,idGoogle,idAmazon
0,http://www.google.com/base/feeds/snippets/1112...,b000i2qubi
1,http://www.google.com/base/feeds/snippets/1112...,b000hc0m36
2,http://www.google.com/base/feeds/snippets/1112...,b000h14rqk
3,http://www.google.com/base/feeds/snippets/1112...,b00062ak2k
4,http://www.google.com/base/feeds/snippets/1112...,b000hbys96
...,...,...
3412,http://www.google.com/base/feeds/snippets/6859...,b000ndo7n0
3413,http://www.google.com/base/feeds/snippets/1362...,b000fico2o
3414,http://www.google.com/base/feeds/snippets/1362...,b000fn6c2c
3415,http://www.google.com/base/feeds/snippets/1362...,b000im5gb8


Nachdem wir die Daten indiziert haben müssen wir überlegen was wir miteinander
vergleichen name mit name vltt beschreibung mit beschreicung Hersteller ...

In [131]:
# Comparsion
compare=rl.Compare()

# Compare name
compare.string("name","name",method="jarowinkler",threshold=0.85,label="cm_name")

# Compare description
compare.string('manufacturer', 'manufacturer', method='jarowinkler',
                threshold=0.85,label="cm_manufacturer")

# Compare Price
compare.numeric("price","price",method="linear",label="cm_price")

# Compute comparsion vectors
features = compare.compute(candidate_links, goog, amzn)

features

cm_name  \
idGoogle                                           idAmazon              
http://www.google.com/base/feeds/snippets/11125... b000i2qubi      0.0   
                                                   b000hc0m36      0.0   
                                                   b000h14rqk      0.0   
                                                   b00062ak2k      0.0   
                                                   b000hbys96      0.0   
...                                                                ...   
http://www.google.com/base/feeds/snippets/68599... b000ndo7n0      0.0   
http://www.google.com/base/feeds/snippets/13620... b000fico2o      0.0   
                                                   b000fn6c2c      0.0   
                                                   b000im5gb8      0.0   
                                                   b000fn6cf4      0.0   

                                                               cm_manufacturer  \
idGoogle                                           idAmazon                      
http://www.google.com/base/feeds/snippets/11125... b000i2qubi              1.0   
                                                   b000hc0m36              1.0   
                                                   b000h14rqk              1.0   
                                                   b00062ak2k              1.0   
                                                   b000hbys96              1.0   
...                                                                        ...   
http://www.google.com/base/feeds/snippets/68599... b000ndo7n0              0.0   
http://www.google.com/base/feeds/snippets/13620... b000fico2o              1.0   
                                                   b000fn6c2c              1.0   
                                                   b000im5gb8              1.0   
                                                   b000fn6cf4              1.0   

                                                               cm_price  
idGoogle                                           idAmazon              
http://www.google.com/base/feeds/snippets/11125... b000i2qubi      0.00  
                                                   b000hc0m36      0.52  
                                                   b000h14rqk      0.00  
                                                   b00062ak2k      0.00  
                                                   b000hbys96      0.00  
...                                                                 ...  
http://www.google.com/base/feeds/snippets/68599... b000ndo7n0      0.00  
http://www.google.com/base/feeds/snippets/13620... b000fico2o      0.00  
                                                   b000fn6c2c      0.00  
                                                   b000im5gb8      0.00  
                                                   b000fn6cf4      0.00  

[3417 rows x 3 columns]

In [133]:
prediction=features[features.sum(axis=1)>1.5]
prediction.shape

(259, 3)

In [108]:
mapping=pd.read_parquet(r"data\Amazon_Google_perfectMapping")

In [134]:
mapping.shape

(1300, 2)

In [123]:
matches=mapping.set_index(["idGoogle","idAmazon"])

In [124]:
matches.index

MultiIndex([('http://www.google.com/base/feeds/snippets/18441480711193821750', ...),
            ('http://www.google.com/base/feeds/snippets/18441110047404795849', ...),
            ('http://www.google.com/base/feeds/snippets/18441188461196475272', ...),
            ('http://www.google.com/base/feeds/snippets/18428750969726461849', ...),
            ('http://www.google.com/base/feeds/snippets/18430621475529168165', ...),
            ('http://www.google.com/base/feeds/snippets/18434163358494173665', ...),
            ('http://www.google.com/base/feeds/snippets/18363672170449359273', ...),
            ('http://www.google.com/base/feeds/snippets/18431790997623472871', ...),
            ('http://www.google.com/base/feeds/snippets/18434733520229761009', ...),
            ('http://www.google.com/base/feeds/snippets/18434262004669557918', ...),
            ...
            ('http://www.google.com/base/feeds/snippets/18037480601440105621', ...),
            ( 'http://www.google.com/base/feeds/s

In [135]:
# Get the confusion matrix
confusion_matrix=rl.confusion_matrix(prediction,matches,len(features))

print(confusion_matrix)


print("Precision:",rl.precision(confusion_matrix))
print("Recall:",rl.recall(confusion_matrix))
print("F-Measure:",rl.fscore(confusion_matrix))

[[  24  235]
 [1276 1882]]
Precision: 0.018461538461538463
Recall: 0.09266409266409266
F-Measure: 0.03078896728672226
